In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F


In [2]:
texts=["hola mundo cruel", "me llamo eduardo", "buenas noches a todos"]

tokens = []
vocabulary = set()
for text in texts:
    toks = text.split()
    tokens.append(toks)
    for w in toks:
        vocabulary.add(w)

print(tokens)

unk_token = "<UNK>"
pad_token = '<PAD>'
print(vocabulary)

print(f"Vocabulary length {len(vocabulary) + 2}")

[['hola', 'mundo', 'cruel'], ['me', 'llamo', 'eduardo'], ['buenas', 'noches', 'a', 'todos']]
{'a', 'hola', 'todos', 'noches', 'llamo', 'buenas', 'cruel', 'eduardo', 'me', 'mundo'}
Vocabulary length 12


In [3]:
word2id = {unk_token:0, pad_token:1}
id2word = {0:unk_token, 1:pad_token}

for i, w in enumerate(vocabulary,2):
    word2id[w] = i
    id2word[i] = w

print(word2id)
print(id2word)

vocabulary_size = len(word2id)
print(f"Vocabulary length {vocabulary_size}")

{'<UNK>': 0, '<PAD>': 1, 'a': 2, 'hola': 3, 'todos': 4, 'noches': 5, 'llamo': 6, 'buenas': 7, 'cruel': 8, 'eduardo': 9, 'me': 10, 'mundo': 11}
{0: '<UNK>', 1: '<PAD>', 2: 'a', 3: 'hola', 4: 'todos', 5: 'noches', 6: 'llamo', 7: 'buenas', 8: 'cruel', 9: 'eduardo', 10: 'me', 11: 'mundo'}
Vocabulary length 12


In [4]:
def numeralize(text):
    sent = []
    for w in text:
        if w in vocabulary:
            sent.append(word2id[w])
        else:
            sent.append(0)  # <UNK>
    return sent

In [5]:
text = ["hola", "lalo"]
print(numeralize(text))

[3, 0]


In [6]:
text_numbers = [numeralize(tok) for tok in tokens]
print(text_numbers)

[[3, 11, 8], [10, 6, 9], [7, 5, 2, 4]]


In [7]:
larger = 0
for text in text_numbers:
    if len(text) > larger:
        larger = len(text)

print(f"Largest text of {larger} words")

Largest text of 4 words


In [8]:
def pad_sentence(text):
    if len(text) < larger:
        diff = larger - len(text)
        for i in range(diff):
            text.append(1)  # add token "<PAD>"
    return text

In [9]:
padded_text = [pad_sentence(text) for text in text_numbers]
print(padded_text)

[[3, 11, 8, 1], [10, 6, 9, 1], [7, 5, 2, 4]]


In [10]:
def textify(text):
    words = []
    for number in text:
        words.append(id2word[number])
    return words

In [11]:
text_again = [textify(text) for text in padded_text]
print(text_again)

[['hola', 'mundo', 'cruel', '<PAD>'], ['me', 'llamo', 'eduardo', '<PAD>'], ['buenas', 'noches', 'a', 'todos']]


In [12]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(Embedder, self).__init__()
        self.embed = nn.Embedding(vocab_size+1, emb_dim)    # +1 for the ids not starting with 0

    def forward(self, x):
        return self.embed(x)

    def print_weights(self):
        return self.embed.weight

In [13]:
embedding_dimension = 5

text1 = padded_text[0]
print(text1)
print(type(text1))

[3, 11, 8, 1]
<class 'list'>


In [14]:
batch = np.vstack(padded_text)
print(batch)
print(type(batch))
print(batch[0])
print(type(batch[0]))

[[ 3 11  8  1]
 [10  6  9  1]
 [ 7  5  2  4]]
<class 'numpy.ndarray'>
[ 3 11  8  1]
<class 'numpy.ndarray'>


In [15]:
input_batch = torch.tensor(batch, dtype=torch.long)
print(input_batch)

tensor([[ 3, 11,  8,  1],
        [10,  6,  9,  1],
        [ 7,  5,  2,  4]])


In [16]:
print(input_batch[0])
print(type(input_batch[0]))

tensor([ 3, 11,  8,  1])
<class 'torch.Tensor'>


In [17]:
embedder = Embedder(vocabulary_size, embedding_dimension)
print(embedder.print_weights())


Parameter containing:
tensor([[ 4.2069e-01,  7.6514e-02, -6.3567e-01,  1.9497e+00, -8.9343e-01],
        [ 6.1225e-01, -1.2635e+00, -2.6804e-01,  6.8436e-01, -6.6422e-01],
        [ 1.9512e+00,  7.0124e-01, -7.4065e-01,  5.1531e-01, -9.4208e-01],
        [-8.8818e-02,  1.1508e+00, -8.0582e-01,  1.8525e+00,  1.1676e+00],
        [ 1.7644e+00,  8.0813e-02,  2.8024e-01,  5.4458e-02, -4.9933e-02],
        [ 2.4747e-01,  1.9355e+00,  1.0355e+00,  1.3789e-01, -2.9034e-01],
        [-4.1499e-01,  5.6145e-01, -9.8728e-01, -2.7918e-01, -3.9903e-01],
        [-1.4724e+00, -6.5922e-01, -9.3738e-01, -1.5431e-01,  5.1570e-01],
        [-5.6930e-01, -8.3106e-04, -5.0901e-01,  1.1157e+00, -4.0423e-01],
        [ 9.4209e-01,  1.5095e-01, -1.5613e+00, -3.1993e-01,  8.3828e-01],
        [-8.6807e-01, -1.6060e-01,  5.2281e-01,  7.1901e-01, -9.9410e-01],
        [-1.2920e+00,  7.8423e-01, -9.0783e-02, -8.5154e-01,  1.3041e-01],
        [ 1.9151e-01,  2.7788e-01, -7.2578e-01, -4.9673e-01,  1.7045e-01]],
  

In [18]:
print(embedder(input_batch))
print(embedder(input_batch).shape)


tensor([[[-8.8818e-02,  1.1508e+00, -8.0582e-01,  1.8525e+00,  1.1676e+00],
         [-1.2920e+00,  7.8423e-01, -9.0783e-02, -8.5154e-01,  1.3041e-01],
         [-5.6930e-01, -8.3106e-04, -5.0901e-01,  1.1157e+00, -4.0423e-01],
         [ 6.1225e-01, -1.2635e+00, -2.6804e-01,  6.8436e-01, -6.6422e-01]],

        [[-8.6807e-01, -1.6060e-01,  5.2281e-01,  7.1901e-01, -9.9410e-01],
         [-4.1499e-01,  5.6145e-01, -9.8728e-01, -2.7918e-01, -3.9903e-01],
         [ 9.4209e-01,  1.5095e-01, -1.5613e+00, -3.1993e-01,  8.3828e-01],
         [ 6.1225e-01, -1.2635e+00, -2.6804e-01,  6.8436e-01, -6.6422e-01]],

        [[-1.4724e+00, -6.5922e-01, -9.3738e-01, -1.5431e-01,  5.1570e-01],
         [ 2.4747e-01,  1.9355e+00,  1.0355e+00,  1.3789e-01, -2.9034e-01],
         [ 1.9512e+00,  7.0124e-01, -7.4065e-01,  5.1531e-01, -9.4208e-01],
         [ 1.7644e+00,  8.0813e-02,  2.8024e-01,  5.4458e-02, -4.9933e-02]]],
       grad_fn=<EmbeddingBackward>)
torch.Size([3, 4, 5])


In [19]:
n_users = 6
n_items = 9

embedder_users = Embedder(n_users, embedding_dimension)
embedder_items = Embedder(n_items, embedding_dimension)

print("User embeddings")
print(embedder_users.print_weights())

print("Item embeddings")
print(embedder_items.print_weights())

User embeddings
Parameter containing:
tensor([[-0.2380, -0.3900, -0.3008, -2.2889,  1.4160],
        [ 0.7804,  0.3699, -0.7864,  0.2914, -0.6794],
        [ 0.4083,  1.8565, -0.6354,  0.3214, -0.5585],
        [-0.1887,  0.7602, -0.2233,  1.2523,  0.5037],
        [ 0.5539, -0.5804,  0.2078,  0.4279, -0.6583],
        [-0.3904,  0.2019,  1.6831, -1.8207,  0.8827],
        [ 0.2987, -0.2708,  0.7202, -0.3698,  1.4666]], requires_grad=True)
Item embeddings
Parameter containing:
tensor([[-0.9171,  1.2775, -1.5247,  0.0356,  0.6333],
        [ 0.6971,  1.2208,  1.6442, -1.8370, -0.5363],
        [-0.1975,  0.8366,  0.4677,  0.1314,  0.0982],
        [ 0.9187, -0.1161,  1.1129,  0.2691,  2.0420],
        [ 0.0883, -0.0461,  1.2943,  0.5682, -0.1644],
        [-0.2421, -0.0471,  1.8228,  0.8298, -1.6756],
        [ 1.4120, -1.3770, -0.0187, -0.4971, -0.3081],
        [ 0.4083,  0.8845,  1.6022,  2.1710,  0.6012],
        [-0.5811,  1.9930,  0.8790,  1.3150, -1.8118],
        [ 0.6352,  1.42

In [20]:
user_item = (1,1)
ratings = [4.5, 2.5, 5.0]
user_1 = torch.tensor(user_item[0], dtype=torch.long)
item_1 = torch.tensor(user_item[1], dtype=torch.long)

In [21]:
user_1_embs = embedder_users(user_1)
print(user_1_embs)
print(user_1_embs.shape)

tensor([ 0.7804,  0.3699, -0.7864,  0.2914, -0.6794],
       grad_fn=<EmbeddingBackward>)
torch.Size([5])


In [22]:
item_1_embs = embedder_items(item_1)
print(item_1_embs)

tensor([ 0.6971,  1.2208,  1.6442, -1.8370, -0.5363],
       grad_fn=<EmbeddingBackward>)


## LSTM

Get first sentence

In [23]:
print(input_batch)
sentence1 = input_batch[0]

print(sentence1)

tensor([[ 3, 11,  8,  1],
        [10,  6,  9,  1],
        [ 7,  5,  2,  4]])
tensor([ 3, 11,  8,  1])


In [24]:
print(sentence1.shape)

torch.Size([4])


In [25]:
def prepare_sequence(seq):
    idxs = [word2id[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

input_ = prepare_sequence(["hola","mundo","cruel", "<PAD>"])
print(input_)
print(input_.shape)

tensor([ 3, 11,  8,  1])
torch.Size([4])


In [26]:
embed = nn.Embedding(vocabulary_size, embedding_dimension)
print(embed(input_))

tensor([[ 0.8454,  0.2955, -1.6030,  2.3140,  1.2802],
        [-0.4422,  0.6616,  1.7816,  0.8376,  1.5056],
        [-0.0143, -0.1195, -0.0478,  0.1282, -1.0189],
        [ 0.3890,  0.2782,  0.6848,  0.0927,  0.7669]],
       grad_fn=<EmbeddingBackward>)


In [27]:
word_embeddings = nn.Embedding(vocabulary_size, embedding_dimension)
embeds = embedder(input_)
print(embeds)
print(embeds.shape)

tensor([[-8.8818e-02,  1.1508e+00, -8.0582e-01,  1.8525e+00,  1.1676e+00],
        [-1.2920e+00,  7.8423e-01, -9.0783e-02, -8.5154e-01,  1.3041e-01],
        [-5.6930e-01, -8.3106e-04, -5.0901e-01,  1.1157e+00, -4.0423e-01],
        [ 6.1225e-01, -1.2635e+00, -2.6804e-01,  6.8436e-01, -6.6422e-01]],
       grad_fn=<EmbeddingBackward>)
torch.Size([4, 5])


In [28]:
hidden_dim = 5
lstm = nn.LSTM(embedding_dimension, hidden_dim)
print("Sentence size:",len(sentence1))
embeds_in = embeds.view(len(sentence1), 1, -1)
print(embeds_in.shape)

Sentence size: 4
torch.Size([4, 1, 5])


In [29]:
lstm_out_, (hn_, cn_) = lstm(embeds_in)
print("lstm_out")
print(lstm_out_)
print(lstm_out_.shape)
print()
print("hn")
print(hn_)
print(hn_.shape)
print("cn")
print(cn_)
print(cn_.shape)

lstm_out
tensor([[[-0.0058, -0.1278,  0.1569, -0.0781,  0.0857]],

        [[-0.1830,  0.0238,  0.0035, -0.0726,  0.0350]],

        [[-0.1541, -0.0381,  0.1317,  0.0651,  0.1740]],

        [[-0.1939,  0.0252,  0.2474,  0.2457,  0.2950]]],
       grad_fn=<StackBackward>)
torch.Size([4, 1, 5])

hn
tensor([[[-0.1939,  0.0252,  0.2474,  0.2457,  0.2950]]],
       grad_fn=<StackBackward>)
torch.Size([1, 1, 5])
cn
tensor([[[-0.3703,  0.0530,  0.4538,  0.4661,  0.5441]]],
       grad_fn=<StackBackward>)
torch.Size([1, 1, 5])


# Using the whole batch

In [30]:
print(input_batch)
print(input_batch.shape)
embeds_batch_in = embedder(input_batch)
print(embeds_batch_in)
print(embeds_batch_in.shape)

tensor([[ 3, 11,  8,  1],
        [10,  6,  9,  1],
        [ 7,  5,  2,  4]])
torch.Size([3, 4])
tensor([[[-8.8818e-02,  1.1508e+00, -8.0582e-01,  1.8525e+00,  1.1676e+00],
         [-1.2920e+00,  7.8423e-01, -9.0783e-02, -8.5154e-01,  1.3041e-01],
         [-5.6930e-01, -8.3106e-04, -5.0901e-01,  1.1157e+00, -4.0423e-01],
         [ 6.1225e-01, -1.2635e+00, -2.6804e-01,  6.8436e-01, -6.6422e-01]],

        [[-8.6807e-01, -1.6060e-01,  5.2281e-01,  7.1901e-01, -9.9410e-01],
         [-4.1499e-01,  5.6145e-01, -9.8728e-01, -2.7918e-01, -3.9903e-01],
         [ 9.4209e-01,  1.5095e-01, -1.5613e+00, -3.1993e-01,  8.3828e-01],
         [ 6.1225e-01, -1.2635e+00, -2.6804e-01,  6.8436e-01, -6.6422e-01]],

        [[-1.4724e+00, -6.5922e-01, -9.3738e-01, -1.5431e-01,  5.1570e-01],
         [ 2.4747e-01,  1.9355e+00,  1.0355e+00,  1.3789e-01, -2.9034e-01],
         [ 1.9512e+00,  7.0124e-01, -7.4065e-01,  5.1531e-01, -9.4208e-01],
         [ 1.7644e+00,  8.0813e-02,  2.8024e-01,  5.4458e-02, 

In [31]:
lstm_out, (hn, cn) = lstm(embeds_in)
print("lstm_out")
print(lstm_out)
print(lstm_out.shape)
print()
print("hn")
print(hn)
print(hn.shape)
print()
print("cn")
print(cn)
print(cn.shape)



lstm_out
tensor([[[-0.0058, -0.1278,  0.1569, -0.0781,  0.0857]],

        [[-0.1830,  0.0238,  0.0035, -0.0726,  0.0350]],

        [[-0.1541, -0.0381,  0.1317,  0.0651,  0.1740]],

        [[-0.1939,  0.0252,  0.2474,  0.2457,  0.2950]]],
       grad_fn=<StackBackward>)
torch.Size([4, 1, 5])

hn
tensor([[[-0.1939,  0.0252,  0.2474,  0.2457,  0.2950]]],
       grad_fn=<StackBackward>)
torch.Size([1, 1, 5])

cn
tensor([[[-0.3703,  0.0530,  0.4538,  0.4661,  0.5441]]],
       grad_fn=<StackBackward>)
torch.Size([1, 1, 5])


## Attention

In [32]:
class Attention(nn.Module):

    def __init__(self, dimensions, attention_type='general'):
        super(Attention, self).__init__()

        if attention_type not in ['dot', 'general']:
            raise ValueError("Invalid attention type selected")

        self.attention_type = attention_type
        if self.attention_type == 'general':
            self.linear_in = nn.Linear(dimensions, dimensions, bias=False)

        self.linear_out = nn.Linear(dimensions * 2, dimensions, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        self.tanh = nn.Tanh()

    def forward(self, query, context):
        """

        :param query: [batch_size, output_length, dimensions]
        :param context: [batch_size, output_length, query_length]
        :return:
        output [batch_size, output_legth, dimensions]
        weights [batch_size, output_length, query_length]
        """
        batch_size, output_len, dimensions = query.size()
        query_len = context.size(1)

        if self.attention_type == 'general':
            query = query.reshape(batch_size * output_len, dimensions)
            query = self.linear_in(query)
            query = query.reshape(batch_size, output_len, dimensions)

        # 1. Compute a score for each encoder state
        attention_scores = torch.bmm(query, context.transpose(1,2).contiguous())

        # Compute weights across every context sequence
        attention_scores = attention_scores.view(batch_size * output_len, query_len)
        print("ATT SCORES")
        print(attention_scores.size())

        # 2. Compute the attention weights
        attention_weights = self.softmax(attention_scores)
        attention_weights = attention_weights.view(batch_size, output_len, query_len)

        # 3. Compute the new context vector S
        mix = torch.bmm(attention_weights, context)

        # 4. Concatenate context vector with output of previous time step
        combined = torch.cat((mix, query), dim=2)
        combined = combined.view(batch_size * output_len, 2 * dimensions)

        #
        output = self.linear_out(combined).view(batch_size, output_len, dimensions)
        output = self.tanh(output)

        return mix, output, attention_weights

In [33]:
print("Users embeddings as Query?")
print(user_1_embs)
print(user_1_embs.shape)

Users embeddings as Query?
tensor([ 0.7804,  0.3699, -0.7864,  0.2914, -0.6794],
       grad_fn=<EmbeddingBackward>)
torch.Size([5])


In [34]:
print(lstm_out_.size(0))
aligned_weights = torch.randn(lstm_out_.size(0))
print(aligned_weights)

print("lstm_out_[0]).squeeze()")
x_ = lstm_out_[0].squeeze()
print(x_)
print(x_.shape)

4
tensor([ 1.0619, -0.9165, -0.2354, -0.6006])
lstm_out_[0]).squeeze()
tensor([-0.0058, -0.1278,  0.1569, -0.0781,  0.0857],
       grad_fn=<SqueezeBackward0>)
torch.Size([5])


### dot attention

Atencion con el usuario?

In [35]:
for i in range(lstm_out_.size(0)):
    aligned_weights[i] = torch.dot(user_1_embs, lstm_out_[i].squeeze())

In [36]:
print(aligned_weights)
print(aligned_weights.shape)
print(aligned_weights.unsqueeze(0))
print(aligned_weights.unsqueeze(0).shape)

tensor([-0.2562, -0.1816, -0.3372, -0.4654], grad_fn=<CopySlices>)
torch.Size([4])
tensor([[-0.2562, -0.1816, -0.3372, -0.4654]], grad_fn=<UnsqueezeBackward0>)
torch.Size([1, 4])


In [37]:
aligned_weights_ = F.softmax(aligned_weights.unsqueeze(0))
print(aligned_weights_)
print(aligned_weights_.shape)
print(torch.sum(aligned_weights_.detach(), dim=1))

tensor([[0.2624, 0.2827, 0.2420, 0.2129]], grad_fn=<SoftmaxBackward>)
torch.Size([1, 4])
tensor([1.])


/media/eduardo/SSD_DATA/Appl ML Projects/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


Atencion con el item?

In [38]:
for i in range(lstm_out_.size(0)):
    aligned_weights[i] = torch.dot(item_1_embs, lstm_out_[i].squeeze())

In [39]:
print(aligned_weights)
print(aligned_weights.shape)
print(aligned_weights.unsqueeze(0))
print(aligned_weights.unsqueeze(0).shape)

tensor([ 0.1955,  0.0218, -0.1504, -0.3072], grad_fn=<CopySlices>)
torch.Size([4])
tensor([[ 0.1955,  0.0218, -0.1504, -0.3072]], grad_fn=<UnsqueezeBackward0>)
torch.Size([1, 4])


In [40]:
aligned_weights_ = F.softmax(aligned_weights.unsqueeze(0))
print(aligned_weights_)
print(aligned_weights_.shape)
print(torch.sum(aligned_weights_.detach(), dim=1))



tensor([[0.3171, 0.2666, 0.2244, 0.1919]], grad_fn=<SoftmaxBackward>)
torch.Size([1, 4])
tensor([1.])


/media/eduardo/SSD_DATA/Appl ML Projects/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


New Context Vector

In [41]:
print(aligned_weights_.unsqueeze(0).shape)
print(lstm_out_.view(1,-1,hidden_dim).shape)

#context_vector = torch.bmm(aligned_weights.unsqueeze(0), encoder_hidden_state.view(1, -1 ,self.hidden_size))
context_vector = torch.bmm(aligned_weights_.unsqueeze(0), lstm_out_.view(1,-1,hidden_dim))

print(context_vector)
print(context_vector.shape)

torch.Size([1, 1, 4])
torch.Size([1, 4, 5])
tensor([[[-0.1224, -0.0379,  0.1277,  0.0176,  0.1322]]],
       grad_fn=<BmmBackward>)
torch.Size([1, 1, 5])


## Concatenate context_vector, user_emb, item_emb

In [42]:
print(context_vector)
print(context_vector.shape)
print(context_vector[0])
print(context_vector[0].shape)
print()
print(user_1_embs.unsqueeze(0).shape)
print(item_1_embs.unsqueeze(0).shape)

tensor([[[-0.1224, -0.0379,  0.1277,  0.0176,  0.1322]]],
       grad_fn=<BmmBackward>)
torch.Size([1, 1, 5])
tensor([[-0.1224, -0.0379,  0.1277,  0.0176,  0.1322]],
       grad_fn=<SelectBackward>)
torch.Size([1, 5])

torch.Size([1, 5])
torch.Size([1, 5])


In [43]:
x = torch.cat((context_vector[0], user_1_embs.unsqueeze(0), item_1_embs.unsqueeze(0)))
print(x)
print(x.shape)

tensor([[-0.1224, -0.0379,  0.1277,  0.0176,  0.1322],
        [ 0.7804,  0.3699, -0.7864,  0.2914, -0.6794],
        [ 0.6971,  1.2208,  1.6442, -1.8370, -0.5363]], grad_fn=<CatBackward>)
torch.Size([3, 5])


In [44]:
print(x.unsqueeze(0))
print(x.unsqueeze(0).shape)


tensor([[[-0.1224, -0.0379,  0.1277,  0.0176,  0.1322],
         [ 0.7804,  0.3699, -0.7864,  0.2914, -0.6794],
         [ 0.6971,  1.2208,  1.6442, -1.8370, -0.5363]]],
       grad_fn=<UnsqueezeBackward0>)
torch.Size([1, 3, 5])


In [45]:
x_ = torch.cat((context_vector[0], user_1_embs.unsqueeze(0), item_1_embs.unsqueeze(0)), dim=1)
print(x_)
print(x_.shape)

tensor([[-0.1224, -0.0379,  0.1277,  0.0176,  0.1322,  0.7804,  0.3699, -0.7864,
          0.2914, -0.6794,  0.6971,  1.2208,  1.6442, -1.8370, -0.5363]],
       grad_fn=<CatBackward>)
torch.Size([1, 15])


In [84]:
x_in = x_.squeeze()
print(x_in)
print(x_in.shape)

tensor([-0.1224, -0.0379,  0.1277,  0.0176,  0.1322,  0.7804,  0.3699, -0.7864,
         0.2914, -0.6794,  0.6971,  1.2208,  1.6442, -1.8370, -0.5363],
       grad_fn=<SqueezeBackward0>)
torch.Size([15])


In [85]:
n_latent = 32
lin1 = nn.Linear(x_in.shape[0], 5)
print(lin1)

Linear(in_features=15, out_features=5, bias=True)


In [86]:
z1 = lin1(x_)
print(z1)
print(z1.shape)

tensor([[ 0.2217,  0.1925,  0.8500, -0.5057,  0.1228]],
       grad_fn=<AddmmBackward>)
torch.Size([1, 5])


In [87]:
out1= F.relu(z1)
print(out1)
print(out1.shape)

tensor([[0.2217, 0.1925, 0.8500, 0.0000, 0.1228]], grad_fn=<ReluBackward0>)
torch.Size([1, 5])


In [88]:
out1 = out1.squeeze()
print(out1.shape)
reg = nn.Linear(out1.shape[0], 1)
yr1 = reg(out1)

torch.Size([5])


In [89]:
print(yr1)
print(yr1.shape)

tensor([-0.1921], grad_fn=<AddBackward0>)
torch.Size([1])


Y adjust

In [90]:
y_range = (0, 5.5)
y_adjust= (y_range[1] - y_range[0]) + y_range[0]
print(y_adjust)

5.5


In [91]:
print(yr1 * y_adjust)

tensor([-1.0567], grad_fn=<MulBackward0>)


Second head

In [92]:
lin2 = nn.Linear(x_in.shape[0], 5)
print(lin2)

Linear(in_features=15, out_features=5, bias=True)


In [93]:
z2 = lin2(x_)
print(z2)
print(z2.shape)

tensor([[-0.6157, -0.5333, -0.8299, -0.1530,  0.0195]],
       grad_fn=<AddmmBackward>)
torch.Size([1, 5])


In [94]:
out2= F.relu(z2)
print(out2)
print(out2.shape)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0195]], grad_fn=<ReluBackward0>)
torch.Size([1, 5])


In [95]:
out2 = out2.squeeze()
print(out2.shape)
reg = nn.Linear(out2.shape[0], 1)
yr2 = reg(out2)

torch.Size([5])


In [96]:
print(yr2)
print(yr2.shape)

tensor([-0.0370], grad_fn=<AddBackward0>)
torch.Size([1])


In [97]:
print(yr2 * y_adjust)


tensor([-0.2032], grad_fn=<MulBackward0>)


## overall rating

In [98]:
print(out1)
print(out1.shape)
print(out2)
print(out2.shape)

tensor([0.2217, 0.1925, 0.8500, 0.0000, 0.1228], grad_fn=<SqueezeBackward0>)
torch.Size([5])
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0195], grad_fn=<SqueezeBackward0>)
torch.Size([5])


In [99]:
o1 = out1.unsqueeze(0)
o2 = out2.unsqueeze(0)
print()
t = torch.cat((o1, o2), dim=0)
print(t)
print(t.shape)


tensor([[0.2217, 0.1925, 0.8500, 0.0000, 0.1228],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0195]], grad_fn=<CatBackward>)
torch.Size([2, 5])


In [100]:
t_ = t[0].squeeze()
print(t_)
print(t_.shape)

tensor([0.2217, 0.1925, 0.8500, 0.0000, 0.1228], grad_fn=<SqueezeBackward0>)
torch.Size([5])


In [101]:
print(user_1_embs.shape)
u_i = torch.mul(user_1_embs, item_1_embs)
print(u_i)
print(u_i.shape)

torch.Size([5])
tensor([ 0.5440,  0.4515, -1.2929, -0.5352,  0.3643], grad_fn=<MulBackward0>)
torch.Size([5])


In [102]:
aligned_weights = torch.randn(t.size(0))
print(aligned_weights)

tensor([-1.2850,  0.8361])


In [103]:
for i in range(t.size(0)):
    aligned_weights[i] = torch.dot(u_i, t[i].squeeze())

In [104]:
print(aligned_weights)
print(aligned_weights.shape)
print(aligned_weights.unsqueeze(0))
print(aligned_weights.unsqueeze(0).shape)

tensor([-0.8467,  0.0071], grad_fn=<CopySlices>)
torch.Size([2])
tensor([[-0.8467,  0.0071]], grad_fn=<UnsqueezeBackward0>)
torch.Size([1, 2])


In [105]:
aligned_weights_ = F.softmax(aligned_weights.unsqueeze(0))
print(aligned_weights_)
print(aligned_weights_.shape)
print(torch.sum(aligned_weights_.detach(), dim=1))

tensor([[0.2986, 0.7014]], grad_fn=<SoftmaxBackward>)
torch.Size([1, 2])
tensor([1.])


/media/eduardo/SSD_DATA/Appl ML Projects/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


New Context Vector para el overall rating

In [106]:
print(aligned_weights_.unsqueeze(0).shape)
print(t.view(1,-1,hidden_dim).shape)

#context_vector = torch.bmm(aligned_weights.unsqueeze(0), encoder_hidden_state.view(1, -1 ,self.hidden_size))
context_vector = torch.bmm(aligned_weights_.unsqueeze(0), t.view(1,-1,hidden_dim))

print(context_vector)
print(context_vector.shape)


torch.Size([1, 1, 2])
torch.Size([1, 2, 5])
tensor([[[0.0662, 0.0575, 0.2538, 0.0000, 0.0503]]], grad_fn=<BmmBackward>)
torch.Size([1, 1, 5])
